### Clone and index all Smart Data Models

**This is an optional step if you are building a FIWARE Smart Data Model solution**

Run the Ansible Playbook below to clone the hundreds of FIWARE Smart Data Models, and index each Smart Data Model in Apache Solr. We search this Smart Data Model data when we create new Java classes to see if there is a `SmartDataModel: search terms...` class comment in the code that closely matches a Smart Data Model in the Solr search engine, then we will generate all the code for this Smart Data Model automatically. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
ansible-playbook -e @$VARS_PATH playbooks/clone-smart-model-data.yaml
echo DONE

In [ ]:
SEARCH_TERM="Weather measurement"
REGEX1='s/( +)/ OR /g'
REGEX2='s/^([\w\W])$/"$1"/g'
ENCODED_STR="$(echo $SEARCH_TERM | perl -p -e 's/([a-z])([A-Z])/$1 $2/g' | perl -p -e 's/( +)/" OR "/g' | perl -p -e 's/^([\w\W]+?)\s*$/("$1")/g')"
echo $ENCODED_STR
DOMAIN_QUERY="domainName_suggested:$ENCODED_STR^2 OR domainName_text_enUS:$ENCODED_STR^5"
SUBMODULE_QUERY="submoduleShortName_suggested:$ENCODED_STR^3 OR submoduleShortName_text_enUS:$ENCODED_STR^10"
MODEL_QUERY="modelName_suggested:$ENCODED_STR^10 OR modelName_text_enUS:$ENCODED_STR^20"
SEARCH_QUERY="$DOMAIN_QUERY OR $SUBMODULE_QUERY OR $MODEL_QUERY"
STUFF="&fl=*,score&sort=score desc"
echo $SEARCH_QUERY

Query the search engine for a smart data model matching the term. 

In [ ]:
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)

curl -k -s -u "$SOLR_USERNAME:$SOLR_PASSWORD" "$SOLR_URL_COMPUTATE"'/query' \
    --data-urlencode 'fl=*,score' \
    --data-urlencode 'sort=score desc' \
    --data-urlencode "fq=computateFiwareSmartDataModel_docvalues_boolean:true" \
    --data-urlencode "q=$SEARCH_QUERY"
echo DONE